<a href="https://colab.research.google.com/github/minnji88/NLP-study/blob/main/3_3_Sentiment_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchtext.data as data
import torchtext.datasets as datasets
import pickle
#print (torch.__version__)

In [3]:
class RNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num):
        super(RNN_Text, self).__init__()
        V = embed_num
        C = class_num
        H = 256
        
        self.embed = nn.Embedding(V, 100)
        self.rnn = nn.LSTM(100,H, bidirectional = True)
        self.out = nn.Linear(H*2, C)
        
    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        ##x = x.unsqueeze(1)  # (N, Ci, W, D)
        x,(_,__) = self.rnn( x, ( self.h, self.c ) )
        
        logit = self.out(x[-1])
        return logit
    def inithidden(self,b):
        self.h = Variable(torch.randn(2, b, 256))
        self.c = Variable(torch.randn(2, b, 256))
        

In [4]:
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')
                #txt= [ d.split('/')[0] for d in line[1].split(' ') ]
                examples += [ data.Example.fromlist( [txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [5]:
text_field = data.Field(fix_length=20)
#text_field = data.Field()
label_field = data.Field(sequential=False, batch_first = True, unk_token = None)

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/자연어처리수업/nsm/small_ratings_train_tok.txt')

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/자연어처리수업/nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)#, device = -1)
len(text_field.vocab)

21893

In [6]:
rnn = RNN_Text(len(text_field.vocab),2)
optimizer = torch.optim.Adam(rnn.parameters())
rnn.train()


RNN_Text(
  (embed): Embedding(21893, 100)
  (rnn): LSTM(100, 256, bidirectional=True)
  (out): Linear(in_features=512, out_features=2, bias=True)
)

In [7]:
%%time
for epoch in range(15):
    z=0
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad()
        
        txt = batch.text
        label = batch.label
        #print (txt.size())
        rnn.inithidden(txt.size(1))
        
        pred = rnn(txt)
        #print(pred.size(), label.size())
        #print(label)
        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward()
        optimizer.step()
        #print(data,label)
        
    print(epoch,'epoch')  
    print(totalloss)
    
torch.save(rnn,'/content/gdrive/My Drive/자연어처리수업/model/rnn_model.pt')

0 epoch
tensor(69.7482)
1 epoch
tensor(65.8978)
2 epoch
tensor(52.4002)
3 epoch
tensor(41.0379)
4 epoch
tensor(32.8586)
5 epoch
tensor(25.6634)
6 epoch
tensor(18.9274)
7 epoch
tensor(13.7942)
8 epoch
tensor(10.1003)
9 epoch
tensor(7.7004)
10 epoch
tensor(5.6907)
11 epoch
tensor(4.9875)
12 epoch
tensor(3.7874)
13 epoch
tensor(3.1730)
14 epoch
tensor(3.0401)
CPU times: user 4min 54s, sys: 4.3 s, total: 4min 58s
Wall time: 4min 59s


In [8]:
%%time
from sklearn.metrics import classification_report
correct = 0
incorrect = 0
rnn.eval()
y_test = []
prediction = []

for batch in test_iter:
    txt = batch.text
    label = batch.label
    y_test.append(label.data[0])
    
    rnn.inithidden(txt.size(1))
    
    pred = rnn(txt)
    _,ans = torch.max(pred,dim=1)
    prediction.append(ans.data[0])
    
    if ans.data[0] == label.data[0]:
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print(classification_report(torch.tensor(y_test), 
                            torch.tensor(prediction), 
                            digits=4, 
                            target_names=['negative', 'positive']))


correct :  87
incorrect :  13
              precision    recall  f1-score   support

    negative     0.8491    0.9000    0.8738        50
    positive     0.8936    0.8400    0.8660        50

    accuracy                         0.8700       100
   macro avg     0.8713    0.8700    0.8699       100
weighted avg     0.8713    0.8700    0.8699       100

CPU times: user 745 ms, sys: 122 ms, total: 868 ms
Wall time: 1.11 s
